# Prompt2Model - Generate Deployable Models from Instructions

[Prompt2Model](https://github.com/neulab/prompt2model) is a system that takes a natural language task description (like the prompts used for large language models such as ChatGPT) to train a small special-purpose model that is conducive for deployment.

In this demo, we demonstrate how to use Prompt2Model to create a model that answers questions over documents, but you can adapt it to any task you like by changing the initial prompt and adjusting the following design decisions appropriately.

Note that every place that has a comment saying `CHANGE THIS` is a variable that you can change to adapt the demo to your task.

<a href="https://colab.research.google.com/github/neulab/prompt2model/blob/main/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you have any questions or feedback, please feel free to contact us!

- **Github:** open an [issue](https://github.com/neulab/prompt2model/issues) or submit a PR
- **Discord:** join us on [discord](https://discord.gg/UCy9csEmFc)
- **Twitter:** reach out to [@vijaytarian](https://twitter.com/vijaytarian) and [@Chenan3_Zhao](https://twitter.com/Chenan3_Zhao)

## Setting Up

First, start out by installing prompt2model from pypi.

In [ ]:
%pip install prompt2model

Set your OpenAI API key as an environment variable. A good way to do this is to create a `.env` file with a single line
```bash
OPENAI_API_KEY=<your key here>
```
and then run the following command to load environment variables from your `.env` file into the running script.

In [ ]:
%pip install python-dotenv
import dotenv
dotenv.load_dotenv()

You can check to make sure that the key is actually imported by printing out the first few characters of it.

```python

In [ ]:
import os
os.environ['OPENAI_API_KEY'][:3]

## Specify your Prompt

The most important design decision in using prompt2model is what prompt you will use to specify your task. In order to do so it is best to:

1. Explain your task
2. Provide a few examples

In this demo, we will use the following prompt to specify a **question answering system**. If you want to try prompt2model on a new task, you can write a similar prompt by swapping in a new description and new examples. Note that this format is a bit flexible, so you don't have to follow this *exact* format, but it is a good starting point. You can also see our suggestions on [writing good prompts](https://github.com/neulab/prompt2model/blob/main/prompt_examples.md).

In [ ]:
# CHANGE THIS if you want to use a different prompt or tackle a different task
prompt = """
Your task is to generate an answer to a natural question. In this task, the input is a string that consists of both a question and a context passage. The context is a descriptive passage related to the question and contains the answer. And the question can range from Math, Cultural, Social, Geometry, Biology, History, Sports, Technology, Science, and so on.

Here are examples with input questions and context passages, along with their expected outputs:

input="Question: What city did Super Bowl 50 take place in? Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."
output="Santa Clara"

input="Question: What river runs through Warsaw? Context: Warsaw (Polish: Warszawa [varˈʂava] ( listen); see also other names) is the capital and largest city of Poland. It stands on the Vistula River in east-central Poland, roughly 260 kilometres (160 mi) from the Baltic Sea and 300 kilometres (190 mi) from the Carpathian Mountains. Its population is estimated at 1.740 million residents within a greater metropolitan area of 2.666 million residents, which makes Warsaw the 9th most-populous capital city in the European Union. The city limits cover 516.9 square kilometres (199.6 sq mi), while the metropolitan area covers 6,100.43 square kilometres (2,355.39 sq mi)."
output="Vistula River"

input="Question: The Ottoman empire controlled territory on three continents, Africa, Asia and which other? Context: The Ottoman Empire was an imperial state that lasted from 1299 to 1923. During the 16th and 17th centuries, in particular at the height of its power under the reign of Suleiman the Magnificent, the Ottoman Empire was a powerful multinational, multilingual empire controlling much of Southeast Europe, Western Asia, the Caucasus, North Africa, and the Horn of Africa. At the beginning of the 17th century the empire contained 32 provinces and numerous vassal states. Some of these were later absorbed into the empire, while others were granted various types of autonomy during the course of centuries."
output="Europe"
"""

## Parse the Prompt

Next, Prompt2Model parses out the instructions an examples from the prompt.
We use the `OpenAIInstructionParser` to do so.

In [ ]:
from prompt2model.prompt_parser import OpenAIInstructionParser, TaskType

prompt_spec = OpenAIInstructionParser(task_type=TaskType.TEXT_GENERATION)
prompt_spec.parse_from_prompt(prompt)
print(f"Instruction:\n{prompt_spec.instruction}\n")
print(f"Examples:\n{prompt_spec.examples}")

## Retrieve and Process Dataset

Next, `Prompt2Model` searches for datasets on Hugging Face to try to find training datasets that may be useful for your task. Specifically, we use `DescriptionDatasetRetriever`, which looks up datasets that match the description.

First we initialize the retriever. This creates the search index so it may take several minutes the first time you run it.

In [ ]:
from prompt2model.dataset_retriever import DescriptionDatasetRetriever

retriever = DescriptionDatasetRetriever()

Next we retriever a list of top datasets for the current prompt (and display their basic data).

In [ ]:

sorted_dataset_list = retriever.retrieve_top_datasets(prompt_spec)

print("#\tName\tDescription")
for i, d in enumerate(sorted_dataset_list):
    description_no_spaces = d.description.replace("\n", " ")
    print(f"{i+1}):\t{d.name}\t{description_no_spaces}")

retrieved_dataset_dict = None

If none of the datasets in the list look useful for your task, you can skip the rest of the section and we won't use any retrieved data.

However, if one of the datasets looks useful, set the `retrieved_dataset_name` variable, and continue through the rest of the section. For the question answering example, we will pick the `squad` dataset to train our model, but of course you will want to change this to the dataset that you selected if you're doing a different task.


In [ ]:
# CHANGE THIS if you want to use a different retrieved dataset
retrieved_dataset_name = "squad"

Existing datasets on Hugging Face have many different formats, but Prompt2Model expects that a dataset should have one input and one output, both of which are strings. In order to solve this, we do a **canonicalization** step, where we convert the dataset into a format that is compatible with `Prompt2Model`.

In order to do so, we examine the dataset and find the different configurations that exist.

In [ ]:
import datasets

configs = datasets.get_dataset_config_names(retrieved_dataset_name)
print(f"Available dataset configs {configs}")

Then we choose one.

In [ ]:
# CHANGE THIS if you want to use a different dataset configuration
chosen_config = "plain_text"

Next, we read in the dataset and print out an example. You can use this to check which columns you'd like to use for the input and output.

In [ ]:
import json

dataset = datasets.load_dataset(retrieved_dataset_name, chosen_config)
if "train" not in dataset:
    raise ValueError(
        f"Dataset {retrieved_dataset_name} does not have a train split."
    )
train_columns = dataset["train"].column_names
train_columns_formatted = ", ".join(train_columns)

if len(dataset["train"]) == 0:
    raise ValueError(
        f"Dataset {retrieved_dataset_name} has no rows in the train split."
    )
example_rows = json.dumps(dataset["train"][0], indent=4)

print(f"Loaded dataset. Example row:\n{example_rows}\n")

print(f"It has these columns: {train_columns_formatted}.")

Now we can set the following variables to the ones that we'd like to use.

In [ ]:
# CHANGE THIS if you want to use a different dataset configuration
input_columns = ["question", "context"]
output_column = "answers"

Finally, we canonicalize the dataset, and we have properly prepared our dataset for training. We also save it to disk.

In [ ]:
retrieved_dataset_dict = retriever.canonicalize_dataset_using_columns(
    dataset, input_columns, output_column
)
retrieved_dataset_dict.save_to_disk("retrieved_dataset_dict")

## Retrieve Model

Use the `DescriptionModelRetriever` to retrieve a pretrain model.

The `top_model_names` is a list of pretrain model names of HuggingFace model. In our demo, we choose the first one as default.

In [ ]:
from prompt2model.model_retriever import DescriptionModelRetriever

retriever = DescriptionModelRetriever(
    model_descriptions_index_path="huggingface_data/huggingface_models/model_info/",  # noqa E501
    use_bm25=True,
    use_HyDE=True,
)
top_model_names = retriever.retrieve(prompt_spec)
pre_train_model_name = top_model_names[0]
print(pre_train_model_name)

## Generate Dataset

Use `OpenAIDatasetGenerator` to generte new examples for the machine reading quesiton-answering task.

In [ ]:
from prompt2model.dataset_generator import OpenAIDatasetGenerator, DatasetSplit

unlimited_dataset_generator = OpenAIDatasetGenerator(
    initial_temperature=0.3,
    max_temperature=1.4,
    responses_per_request=3,
    max_api_calls=10000,
    requests_per_minute=80,
)
generated_dataset = unlimited_dataset_generator.generate_dataset_split(
    prompt_spec, 5000, split=DatasetSplit.TRAIN
)
generated_dataset.save_to_disk("generated_dataset")

## Preprocess Dataset

Combine the `generated_dataset` with `retrieved_dataset_dict` and use `TextualizeProcessor` to preprocess the training dataset.

In [ ]:
import datasets
from prompt2model.dataset_processor import TextualizeProcessor

train_generated_dataset = datasets.Dataset.from_dict(generated_dataset[:3000])
val_generated_dataset = datasets.Dataset.from_dict(generated_dataset[3000: 4000])
test_generated_dataset = datasets.Dataset.from_dict(generated_dataset[4000:])

generated_dataset = datasets.DatasetDict(
    {"train": train_generated_dataset, "val": val_generated_dataset, "test": test_generated_dataset}
)

retrieved_dataset = datasets.DatasetDict(
    {
        "train": datasets.Dataset.from_dict(retrieved_dataset_dict["train"][:3000]),
        "val": datasets.Dataset.from_dict(retrieved_dataset_dict["train"][3000:4000]),
        "test": datasets.Dataset.from_dict(retrieved_dataset_dict["train"][4000:5000]),
    }
)

DATASET_DICTS = [generated_dataset, retrieved_dataset]

t5_processor = TextualizeProcessor(has_encoder=True)
t5_modified_dataset_dicts = t5_processor.process_dataset_dict(
    prompt_spec.instruction, DATASET_DICTS
)

## Finetune the Model

Combine the retrieved dataset with generated dataset and use the `GenerationModelTrainer` to finetune the retrieved model. After the finetuning, we save the model and tokenizer to the disk.

In [ ]:
from prompt2model.model_trainer import GenerationModelTrainer
from prompt2model.utils.logging_utils import get_formatted_logger
from pathlib import Path
import logging

trainer_logger = get_formatted_logger("ModelTrainer")
trainer_logger.setLevel(logging.INFO)
train_datasets = [each["train"] for each in t5_modified_dataset_dicts]
val_datasets = [each["val"] for each in t5_modified_dataset_dicts]
test_datasets = [each["test"] for each in t5_modified_dataset_dicts]

trainer = GenerationModelTrainer(
    pre_train_model_name,
    has_encoder=True,
    executor_batch_size=1,
    tokenizer_max_length=1024,
    sequence_max_length=1280,
)

args_output_root = Path("result/training_output")
args_output_root.mkdir(parents=True, exist_ok=True)

trained_model, trained_tokenizer = trainer.train_model(
    hyperparameter_choices={
        "output_dir": str(args_output_root),
        "save_strategy": "epoch",
        "num_train_epochs": 1,
        "per_device_train_batch_size": 1,
        "evaluation_strategy": "epoch",
    },
    training_datasets=train_datasets,
    validation_datasets=val_datasets,
)

trained_model.save_pretrained("trained_model")
trained_tokenizer.save_pretrained("trained_tokenizer")

## Evaluate Model

After the trainning, we evalaute the trained model on the conbined test set with `ModelEvaluator`.

In [ ]:
from prompt2model.model_executor import GenerationModelExecutor
from prompt2model.model_evaluator import Seq2SeqEvaluator
import transformers
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluator_logger = get_formatted_logger("ModelEvaluator")
evaluator_logger.setLevel(logging.INFO)

trained_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
    "trained_model"
).to(device)
trained_tokenizer = transformers.AutoTokenizer.from_pretrained(
    "trained_tokenizer"
)

test_dataset = datasets.concatenate_datasets(test_datasets)

model_executor = GenerationModelExecutor(
    trained_model,
    trained_tokenizer,
    1,
    tokenizer_max_length=1024,
    sequence_max_length=1280,
)
t5_outputs = model_executor.make_prediction(
    test_set=test_dataset, input_column="model_input"
)
evaluator = Seq2SeqEvaluator()
metric_values = evaluator.evaluate_model(
    test_dataset,
    "model_output",
    t5_outputs,
    encoder_model_name="xlm-roberta-base",
)
print(metric_values)